1. Cargar el data set
1. Hacer el analisis de forma, cantidades de nulos, value_count de tipo de propiedad etc
1. Agrupar por provincia para que se vea que capital tiene mas datos
1. Value_counts de CABA
1. Desglosar la columna parent_name y tomar barrio (Palermo en vez de Palermo Soho, etc)
1. Limpiar los datos de barrios
1. Nuevos dataframes: Buenos Aires + el analisis de parent_name
1. Chequear tasa de cambio de dolares a pesos con las respectivas pruebas de que este ok calculo
1. Crear una variable/campo/columna nueva con el precio en dolares (calculado como precio en pesos/ tasa de cambio)
1. Chequear nulos con umbral +9 y dropear 
1. Chequear la moneda y dropear los casos extraños
1. Agrupar por barrio y .describe() sobre precio_mt2
1. Box plot y ver outliers o casos llmativos. Graficar los que tengan mayor desvío
1. TOMAR DECISION: de acuerdo al analisis como imputamos los valores faltantes de precio por mt2
    1. Aplicar la media, mediana?
    1. Aplicar el valor del vecino mas cercano en funcion de la cantidad de metros (agrupar por barrio y ordenar por cantidad de metros totales)
    1. Hacer qcut o cut por cantidad de metros, aplicar la media sobre los intervalos  a los deptos que entren en esa cantidad de metros (esto es posible???)
1. Describe de nuevo para ver como cambian los valores con imputacion
1. Analisis del campo descrition: para ambientes y amemities (pileta/piscina, SUM, laundry/lavadero, cochera/garage, Gimnasio/gym, seguridad)
1. Convertir en dummies los valores del campo descripcion y tipo de propiedad
1. Grafico de distribucion con geolocalizacion / cantidad de ventas por barrio segun tipo de propiedad



**Notas**:
- En este momento hay muchas celdas que tiran error porque los objetos tienen nombres distintos! 
- Existe una forma de parsear URLs [URL parser](https://docs.python.org/3/library/urllib.parse.html).<br>
Lo nombramos pero no profundizamos. No sé si tiene mucho sentido adentrarnos en ese analisis y vamos a usar el campo description 
- Traté de ordenar todo lo que ya estaba hecho como lo pusimos en la lista. Lo de la distribucion de MONEDA me quedo antes pero se puede pasar abajo sin problemas. Tambien todo lo que refiere al calculo de tasa de cambio y de precio_uss podria hacerse tanto antes de filtrat caba como despues.
- Una vez que hayamos completado la nb, hay que chequear consistenecia en nombres de variables, objetos, etc etc
- Tambien habria que dar una leida a todos los textos del markdown, dar consistencia a la sinxtaxis (markdown o html) y completar las explicaciones o comentarios del codigo que hagan falta
- Armar el table of contents del archivo (Jazmin)
- Todo lo que no supe como hacer y ya no me daba la cabeza lo deje comentado para retomar despues

<h1>Desarrollo del Desafio 1</h1>

<p>La inmobiliaria Properati publica periódicamente información sobre ofertas de propiedades para venta y alquiler. Ud. deberá asesorar a la inmobiliaria a desarrollar un modelo de regresión que permita predecir el precio por metro cuadrado de una propiedad. El objetivo             
final es que el modelo que desarrollen sea utilizado como tasador automático a ser aplicados a las próximas propiedades que sean comercializadas por la empresa. Para ello la empresa le provee de un dataset correspondiente al primer semestre de 2017.</p>

El dataset es de tamaño entre pequeño y mediano, pero tiene dos complejidades a las que deberá prestarle atención:
 
<ul>
    <li>Peso de missing data en algunas variables relevantes.</li>
    <li>Será importante tener en cuenta el problema de la influencia espacial en los precios por metro cuadrado. En efecto, es probable que existan diferencias importantes de en las diferentes geografías, barrios y zonas analizadas.</li>
</ul>

<h2>Objetivos:</h2>

<ul>
    <li>Efectuar una limpieza del dataset provisto. Particularmente, deberá diseñar estrategias para lidiar con los datos perdidos en ciertas variables.</li>
    <li>Realizar un análisis descriptivo de las principales variables.</li>
    <li>Crear nuevas columnas a partir de las características dadas que puedan tener valor predictivo.</li>
</ul>

### Importar las librerias que vamos a utilizar ###

In [ ]:
import numpy as np
import pandas as pd
import chardet
import re

### 1. Cargar el data set y verificar que los datos se hayan importado correctamente ###

In [ ]:
# Funcion especial para autodetectar el encoding. La eliminamos porque demora mucho!!!
# def get_encoding_type(csv_path):
#     rawdata = open(csv_path, 'rb').read()
#     result = chardet.detect(rawdata)
#     return result.get('encoding')

In [ ]:
#Se define la ruta de la información.
data_propiedades = "../Data/properati.csv"
#Leemos los datos del archivo
data = pd.read_csv(data_propiedades, sep=",", encoding="UTF-8")
#Chequeamos que los datos se hayan importado correctamente
data.head(3)

In [ ]:
data.tail()

### 2. Hacer el analisis de forma, cantidad de registros, cantidades de nulos por campo, value_counts ###

In [ ]:
#Chequeamos cantidad de registros y cantidad de variables
data.shape

In [ ]:
#Vemos qué tipos de datos tienen los campos
data.dtypes

In [ ]:
#renombramos la columba "Unnamed 0"
data.rename(columns={'Unnamed: 0': "Index"}, inplace=True)
data.head(3)

#### Calculos de cantidad de nulos ####

In [ ]:
#Calculamos cantidad de nulos por campo
cant_nulos_por_campo = data.isnull().sum()
cant_nulos_por_campo

In [ ]:
#Calculamos porcentaje de nulos por campo
cant_registros = data.shape[0]
#print(cant_registros)
#len(data.index) es lo mismo

porc_nulos_por_campo = np.round((100 * cant_nulos_por_campo / cant_registros),2)
print(porc_nulos_por_campo)

#### Observamos la distribución de distintos campos que pueden aportar al analisis ####

In [ ]:
#Observamos distribucion de elementos en campo property_type y en el campo operation
property_type_count = data.property_type.value_counts()
property_type_count

property_operation_count = data.operation.value_counts()
property_operation_count

In [ ]:
#Observamos que todos los registros corresponden a Argentina
country_name_count = data.country_name.value_counts()
country_name_count

#Todos los valores de la columna country_name son Argentina 
#de la columna place_with_parents_name se puede reemplazar Argentina por string vacio

len(data.place_with_parent_names.index)
# data.place_with_parent_names.sample(30)

### Distribucion del tipo de moneda ###

In [ ]:
data.currency.value_counts()

In [ ]:
# Creo una mask para traer esos valores.
currency_PEN_mask = data.currency == 'PEN'
data.loc[currency_PEN_mask, :]

In [ ]:
# Creo una mask para traer esos valores.
currency_UYU_mask = data.currency == 'UYU'
data.loc[currency_UYU_mask, :]

#### Visto que no se puede garantizar que estos datos estén cargados correctamente y son solo tres registros en todo el data set, se toma la decision de eliminarlos ####
Chequear que este codigo para dropear esté correcto! 

In [ ]:
# data.drop(data.loc[currency_PEN_mask].index, inplace = True)
# data.drop(data.loc[currency_UYU_mask].index, inplace = True)

### 3. Distribucion por provincia ###
**Nota**: aqui modifico lo que hizo Ignacio porque no creo todavia un dataframe nuevo

In [ ]:
data.state_name.value_counts()

In [ ]:
# data_precio.place_name.value_counts()

### calcular cantidad de nulos por campo por grupo? ###

### 4.5.6. Limpieza de datos de los barrios de caba y counts ###
Haremos un analisis de Capital Federal unicamente, ya que es un grupo que se distingue del resto por la cantidad y tipo de datos (en **place_with_parent_name**)

#### Generamos un nuevo data frame solo con los datos de CABA ####

In [ ]:
data_caba_mask = data.state_name == 'Capital Federal'

data_caba = data.loc[data_caba_mask, :]

data_caba

Reseteamos el index del nuevo data_frame

In [ ]:
data_caba.reset_index()


#### Vemos la distribucion por barrio ####

In [ ]:
data_caba.place_name.value_counts()

In [ ]:
# print(precios_caba.count() / precios_caba.shape[0]) 
print('% de valores incompletos por columna:')
print(f'{round(data_caba.isnull().sum()/data_caba.count(),2)}')
print(' Cantidad de valores incompletos por columna:')
print(data_caba.isnull().sum())

#### Limpiar el campo **place_with_parent_name** ####

1. Eliminar "Argentina" y "|" de los strings

In [ ]:
#Creo una serie sobre la columna para trabajarla
serie_place_with_parent_names = data_caba.place_with_parent_names
serie_place_with_parent_names

In [ ]:
#Primero sacamos el string el pais
pattern_arg = '^\|Argentina\|'
#RE busca |Argentina al comienzo del string
pattern_arg_regex = re.compile(pattern_arg)
#creo una variable nueva para hacer el apply y reemplazo "|Argentina" por string vacio 
place_parent_name_no_arg = serie_place_with_parent_names.apply(lambda x: pattern_arg_regex.sub("", x))

#chequeo los valores que quedaron
place_parent_name_no_arg.head()

In [ ]:
#Elimino los pipes innecesarios
pattern_pipe_final = '\|$'
#RE busca | al final del string
regex_pipe_final = re.compile(pattern_pipe_final)
#Reemplazo el pipe por string vacio así despues puedo splitear la cadena
place_parent_final = place_parent_name_no_arg.apply(lambda x: regex_pipe_final.sub("", x))
place_parent_final

2. Splitear la cadena para obtener una lista

In [ ]:
serie_list_place_split = place_parent_final.apply(lambda x: x.split("|"))
serie_list_place_split.head(10)
# type(serie_list_place_split)

In [ ]:
#Evaluo las longitudes de las listas que me quedaron en la serie
serie_len = serie_list_place_split.apply(len)
serie_len.value_counts()
# serie_len.values

Vemos que hay valores en esa serie que tienen un solo elemento, los miramos para ver si sirven al análisis

In [ ]:
#hacemos una mascara e indexamos la lista a traves de una lamba que compara la longitud de cada elemento
mask_len_1 = serie_list_place_split.apply(lambda x: len(x) == 1)
serie_list_place_split.loc[mask_len_1]

In [ ]:
# indexamos la lista para que nos devuelva los valores con 1 elementos distinto a 'Capital Federal'
mask_len_1_not_caba = serie_list_place_split.apply(lambda x: (len(x) == 1) & (x[0] != 'Capital Federal'))
serie_list_place_split[mask_len_1_not_caba]

Observamos que no todos estos elementos contienen la especificidad del **Barrio** que es necesaria para hacer los calculos de mt2 por barrio 
### dropeamos esos registros ###

In [ ]:
# place_clean = serie_list_place_split.drop
# place_clean


#RECORDAR CAMBIAR LOS OBJETOS EN LOS QUE SIGUE

3. Obtengo dos series de barrio y sub_barrio

In [ ]:
#funciones para extraer el elemento que me interesa de cada lista según la posicion
#también considero la opcion de Capital Federal, donde la logica es distinta

def get_sub_barrio(x):
    return x[2] if len(x) == 3 else np.NaN
        
def get_barrio(x):
    return x[1] if (len(x) == 3 or len(x) == 2) else np.NaN

# def get_provincia(x):
#     return np.NaN if len(x) == 1 else x[0]

In [ ]:
serie_sub_barrio = serie_list_place_split.apply(get_sub_barrio)

serie_barrio = serie_list_place_split.apply(get_barrio)

# serie_provincia = serie_lista_place_split.apply(get_provincia)

In [ ]:
barrio_count = serie_barrio.value_counts()
barrio_count.sort_index()

### 7. Armamos un nuevo DF con CABA y los datos de los barrios sacando las columnas que no necesitamos ###

In [ ]:
#armo un df nuevo para ver cómo quedó la separación
frame = {'Barrio': serie_barrio, 'Sub_Barrio': serie_sub_barrio}  
caba_place_names = pd.DataFrame(frame) 

data_caba_barrios = pd.concat([data_caba, caba_place_names], axis=1, sort=False)
data_caba_barrios.head(3)

#falta hacer filtro de columnas que no sirven

### 8. Chequear tasa de cambio de dolares a pesos con las respectivas pruebas de que este ok calculo ###
### 9. Creacion de columna de precio en dolares ### 

In [ ]:
data_caba_barrios['usdtolocal'] = data_caba_barrios.price_aprox_local_currency / data_caba_barrios.price_aprox_usd
cambio_usd_max = data_caba_barrios.usdtolocal.max()
cambio_usd_min = data_caba_barrios.usdtolocal.min()

print(data_caba_barrios.sample(3))

# Verifico si en el calculo el valor max y min son iguales.
print(round(cambio_usd_max, 2))
print(round(cambio_usd_min, 2))

check_change = round(data_caba.usdtolocal, 2) == round(cambio_usd_max, 2)

print(f'Son todos igual a {round(cambio_usd_max, 2)}: {all(check_change)}')

print(f'Hay valores que son NaN?: {any(data_caba_barrios.usdtolocal.isnull())}')

Revisando tenemos Price y Price_aprox_usd que parecen ser lo mismo ¿? vamos a tratar de compararlos.

In [ ]:
# revisar price contra price_aprox_usd
check_prices_usd = data_caba_barrios.price == data_caba_barrios.price_aprox_usd
print(f'El valor de price es igual a price_aprox_usd?: {all(check_prices_usd)}')
print(f'Hay precios con campos NaN? : {any(data_caba_barrios.price.isnull())}') # Hay precios que son Nulos.

Con esto detectamos que el precio y precio aproximado parecen no ser lo mismo tampoco.

In [ ]:
#creamos una columna con el valor en dolares en funcion de price
data_caba_barrios['price_usd']= round(data_caba_barrios.apply(lambda x: x['price']  if x['currency'] == 'USD' else (x['price'] * x['usdtolocal'] if x['currency'] == 'ARS' else np.NaN), axis=1),2)
data_caba_barrios.sample(10)

Se puede observar que falta completar el **price_per_m2** en la moneda local. Pero!! se puede calcular en dolares la superficie cubierta porque tenemos el precio en USD por m2, **dividiendo el valor por el precio por metro cuadrado, obtendriamos el dato surface_covered_in_m2**

### 10. Chequeamos cantidad de nulos y dropeamos segun el umbral ###

In [ ]:
# print(precios_caba.count() / precios_caba.shape[0]) 
print('% de valores incompletos por columna:')
print(f'{round(data_caba_barrios.isnull().sum()/data_caba_barrios.count(),2)}')
print(' de valores incompletos por columna:')
print(data_caba_barrios.isnull().sum())

In [ ]:
# De los registros de CABA a cuantos les falta más de 10 datos por fila

precios_caba_mas10_mask = data_caba_barrios.isnull().sum(axis=1) > 9 

print(data_caba_barrios.loc[precios_caba_mas10_mask, :])

# Se puede observar que de estos datos no tenemos tanta información como para incluir y los podemos eliminar del set de datos!!
# Existe Palermo Chico ??  no tiene geonames_id ??

# En el caso de optar por eliminar podemos usar: 
# umbral = 9
# precios_caba_clean1 = data_caba.dropna(axis = 0, thresh=umbral)

<strong>Verifico Palermo Chico lo elimiaría si no puedo calcular el precio por metro cuadrado!! Ya que no aportan mucha información!</strong>
<strong>Se puede mejorar con algun grafico?<strong>

In [ ]:
data_caba_geo_mask = data_caba_barrios.geonames_id.isnull()

data_caba_geo = data_caba_barrios.loc[data_caba_geo_mask, :]

data_caba_geo.head()

# Parece que se puede completar sin problemas, aunque no veo el sentido. 

In [ ]:
precios_caba_place_mask = data_caba_barrios.place_name == 'Las Cañitas'
precios_caba_las_cañitas = data_caba_barrios.loc[precios_caba_place_mask, :]

precios_caba_las_cañitas.geonames_id.isnull().sum() / precios_caba_las_cañitas.shape[0]

# Lo que me dice que esta locación no tiene geonames_id !! revisar si se puede analizar automaticamente para el resto.
# Las Cañitas es la denominación informal y no oficial de un sector de unas 20 manzanas dentro del barrio de Palermo en la Ciudad Autónoma de Buenos Aires.

### 11. Tipo de moneda. Quedo como parte del punto 2 #

### 12. Agrupar por barrio y describe() por mt2 ### 

In [ ]:
barrio_group = data_caba_barrios.groupby('Barrio')

In [ ]:
barrio_group.price_per_m2.describe()

### 13. Box plot para ver outliers de los barrios y eliminar o ajustar antes de imputar datos ###

In [ ]:
#Ver cuales outliers de cada barrio y analizar el desvio ANTES de impurtar los mt2 promedio por barrio

### 14. Imputacion de datos faltantes ### Tomar una decision

In [ ]:
# Analicemos este unico registro. 
data_caba_barrios.loc[44469]

# Encontramos que no tiene el precio_usd_per_m2, y se puede calcular...de dos maneras:

# 1. price_usd / surface_covered_in_m2

# Al parecer cuando es un departamente no tiene surface_total_in_m2 !!

In [ ]:
# Calculo el valor del metro cuadrado para esta propiedad...

round(data_caba_barrios.loc[44469].price_usd / data_caba_barrios.loc[44469].surface_covered_in_m2, 2)

# Me da el mismo valor de price_per_m2 que debería estar el pesos! no en USD.

In [ ]:
# Contar todos los elementos de CABA que no tiene price_usd_per_m2

price_usd_per_m2_mask = data_caba_barrios.price_usd_per_m2.isnull()
print(price_usd_per_m2_mask.loc[44469]) # Verifico que la funcion de True para el valor que tengo.
price_usd_per_m2_isnan = data_caba_barrios.loc[price_usd_per_m2_mask, :]

# print(price_usd_per_m2_mask)
print(price_usd_per_m2_mask.value_counts()) # Me deberia dar 0 
print(f'{round(price_usd_per_m2_mask.sum() / data_caba_barrios.shape[0], 2)} % de valores con NaN')

# De esto ultimo se desprende que tenemos 27% de datos donde falta el valor de precio_usd_per_m2

price_usd_per_m2_isnan.sample(10)

Entonces ahora trato de reemplazar los valores faltantes operando con los datos que tengo.

In [ ]:
data_caba_barrios.price_usd_per_m2 = data_caba_barrios.price_usd_per_m2.replace(np.NaN, data_caba_barrios.price_usd / data_caba_barrios.surface_covered_in_m2)

# Investigar el error de abajo, y este proceso demora mucho tiempo.

#### Calcular precio promedio por barrio ####

In [ ]:
# precios_caba.price_per_m2 = precios_caba.price_per_m2.replace(precios_caba.groupby('place_name').price_per_m2.transform(lambda x: x.fillna(x.mean())))
# print(precios_caba.price_per_m2.replace(precios_caba.groupby('place_name').price_per_m2.transform(lambda x: x.fillna(x.mean()))))

# precios_caba.price_per_m2 = precios_caba.groupby('place_name').price_per_m2.transform(lambda x: x.fillna(x.mean()))
# precios_caba.isnull().sum() 

#Parece que funciono!!!!

### 15. Describe() de los grupos nuevamente para ver como quedaron los datos estadisticos

### 16. Analisis del campo **description** para sacar cantidad de ambientes y amenities (si es posible) ###

In [ ]:
description_series = data_caba_barrios['description']
description_series.sample(10)


#### Sacamos cantidad de ambientes a traves de expresiones regulares ####

In [ ]:
amb1_pattern = "((?P<numero_amb>\d)(\s)?(amb|AMB))"
amb1_pattern_regex = re.compile(amb1_pattern)

In [ ]:
amb1_match = description_series.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      amb1_pattern_regex.search(x))
mask_amb1_match_notnull = amb1_match.notnull()

data_caba_barrios.loc[mask_amb1_match_notnull, 'number_rooms'] = \
amb1_match[mask_amb1_match_notnull].apply(lambda x: x.group("numero_amb"))

In [ ]:
data_caba_barrios['number_rooms'].sample(15)

In [ ]:
data_caba_barrios['number_rooms'].dtypes
data_caba_barrios['number_rooms'].notnull().sum()

In [ ]:
dorm1_pattern = "((?P<numero_dorm>\d)(\s)?(dorm|DORM))"
dorm1_pattern_regex = re.compile(dorm1_pattern)

In [ ]:
dorm1_match = description_series.apply(lambda x: x if (x is np.NaN) | (x is None) else dorm1_pattern_regex.search(x))

mask_dorm1_match_notnull = dorm1_match.notnull()

data_caba_barrios.loc[mask_dorm1_match_notnull, 'number_rooms'] = dorm1_match[mask_dorm1_match_notnull].apply(lambda x: x.group("numero_dorm"))
#se asume que el numero de ambientes es numero_dorm +1 FALTA SUMAR UNO A ESE NUMERO (ver ejemplo de century en Clase 5)

In [ ]:
data_caba_barrios['number_rooms'].head(15)


In [ ]:
data_caba_barrios['number_rooms'].notnull().sum()

In [ ]:
#Falta Ambiente unico, mono(\s)?ambiente, 1 y 1/2 amb / dos/tres/cuatro/cinco ([a-zA-Z]*)? dormi

#PISOS: \d piso, planta baja, PH = planta baja, \der\spiso, con palabras?

Amemities (pileta/piscina, SUM, laundry/lavadero, cochera/garage, Gimnasio/gym, seguridad)

### 17. Convertir en dummies los valores del campo descripcion y tipo de propiedad ###


### 18. Grafico de distribucion con geolocalizacion / cantidad de ventas por barrio segun tipo de propiedad ###